In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("C:/Users/RohanRVC/Documents/Business/E-books/E_Books_Unknown_AI/Book/Dataset_collection/Updated_dataset/Final_data1.csv")

In [3]:
df.head()

,Title,Genre,Content,Language
0,The Declaration of Independence of the United ...,History,All of the original files are included in the ...,English
1,The King James Version of the Bible,Religion and Spirituality,Exodus The Third Book of Moses: Called Levitic...,English
2,The Complete Works of William Shakespeare,Drama and Performing Arts,William Shakespeare Contents THE SONNETS ALL’S...,English
3,The Magna Carta,Linguistics and Languages,preparer of the 0.1 version. some of which wer...,English
4,Apocolocyntosis,Religion and Spirituality,"H. D. Rouse Release date : November 1, ROUSE, ...",English


## Text Generation

In [4]:
import random, pickle , numpy as np
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import LSTM,Dense,Activation 
## LSTM is RNN 
from tensorflow.keras.optimizers import RMSprop

In [5]:
df.shape

(11345, 4)

In [6]:
df.dropna(inplace=True)

In [7]:
df.shape

(11332, 4)

In [8]:
text=list(df.Content.values)
joined_text=" ".join(text)

In [9]:
import psutil

# Get memory details
memory = psutil.virtual_memory()

# Total memory
total_memory = memory.total / (1024 * 1024)  # Convert to MB
print(f"Total Memory: {total_memory} MB")

# Available memory
available_memory = memory.available / (1024 * 1024)  # Convert to MB
print(f"Available Memory: {available_memory} MB")


Total Memory: 16169.40234375 MB
Available Memory: 524.4921875 MB


### Storing text on the partial , cause most of the time we won't have enough Ram for it 

In [9]:
len(joined_text)

3784140608

In [10]:
partial_text=joined_text[:100000]

In [11]:
tokenizer=RegexpTokenizer(r"\w+")
tokens=tokenizer.tokenize(partial_text.lower())

In [73]:
len(tokens)

19611

In [12]:
import pickle
with open('C:/Users/RohanRVC/Documents/Business/E-books/E_Books_Unknown_AI/text_generation/Model/model_stage_2/unique_tokens.pkl', 'rb') as f:
        unique_tokens = pickle.load(f)

with open('C:/Users/RohanRVC/Documents/Business/E-books/E_Books_Unknown_AI/text_generation/Model/model_stage_2/unique_token_index.pkl', 'rb') as f:
        unique_token_index = pickle.load(f)

In [55]:
unique_token=np.unique(tokens)

In [13]:
len(unique_tokens)

1853239

In [57]:
unique_token_index={token:idx for idx , token in enumerate(unique_token)}

In [58]:
# how many words do we need to look at before to predict last word


In [14]:
n_words=10
input_words=[]
next_words=[]

In [15]:
for i in range(len(tokens)-n_words):
    input_words.append(tokens[i:i+n_words])
    next_words.append(tokens[i+n_words])

In [18]:
X=np.zeros((len(input_words),n_words,len(unique_tokens)),dtype=bool)
y=np.zeros((len(next_words),len(unique_tokens)),dtype=bool) 

MemoryError: Unable to allocate 329. GiB for an array with shape (19064, 10, 1853239) and data type bool

In [62]:
for i , words in enumerate(input_words):
    for j , word in enumerate(words):
        X[i,j,unique_token_index[word]]=1
        y[i,unique_token_index[next_words[i]]]=1

In [22]:
model=Sequential()
model.add(LSTM(128,input_shape=(n_words,len(unique_token)),return_sequences=True))
model.add(LSTM(128))#,input_shape(n_words,len(unique_token)),return_sequences=True))
model.add(Dense(len(unique_token)))
model.add(Activation('softmax'))

In [23]:
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(learning_rate=0.01),metrics=['accuracy'])


Epoch 1/10
149/149 [==============================] - 15s 75ms/step - loss: 5.9262 - accuracy: 0.0931
Epoch 2/10
149/149 [==============================] - 11s 74ms/step - loss: 5.5084 - accuracy: 0.1350
Epoch 3/10
149/149 [==============================] - 11s 75ms/step - loss: 5.1028 - accuracy: 0.1728
Epoch 4/10
149/149 [==============================] - 10s 70ms/step - loss: 4.7179 - accuracy: 0.2100
Epoch 5/10
149/149 [==============================] - 11s 72ms/step - loss: 4.4156 - accuracy: 0.2310
Epoch 6/10
149/149 [==============================] - 11s 72ms/step - loss: 4.1557 - accuracy: 0.2569
Epoch 7/10
149/149 [==============================] - 10s 70ms/step - loss: 3.9178 - accuracy: 0.2823
Epoch 8/10
149/149 [==============================] - 10s 69ms/step - loss: 3.6771 - accuracy: 0.3086
Epoch 9/10
149/149 [==============================] - 10s 70ms/step - loss: 3.4421 - accuracy: 0.3390
Epoch 10/10
149/149 [==============================] - 11s 73ms/step - loss: 3.187

In [49]:
model.fit(X,y,batch_size=128,epochs=10,shuffle=True)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'while/MatMul' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "c:\Users\RohanRVC\Documents\Business\E-books\E_Books_Unknown_AI\Book\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\RohanRVC\Documents\Business\E-books\E_Books_Unknown_AI\Book\Lib\site-packages\traitlets\config\application.py", line 1046, in launch_instance
      app.start()
    File "c:\Users\RohanRVC\Documents\Business\E-books\E_Books_Unknown_AI\Book\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
      self.io_loop.start()
    File "c:\Users\RohanRVC\Documents\Business\E-books\E_Books_Unknown_AI\Book\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\RohanRVC\AppData\Local\Programs\Python\Python311\Lib\

In [74]:
model.input_shape

(None, 10, 2299)

In [37]:
model.save("C:/Users/RohanRVC/Documents/Business/E-books/E_Books_Unknown_AI/text_generation/Model/model_stage_1/mymodel.h5")

c:\Users\RohanRVC\Documents\Business\E-books\E_Books_Unknown_AI\Book\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [36]:
model=load_model("C:/Users/RohanRVC/Documents/Business/E-books/E_Books_Unknown_AI/text_generation/Model/model_stage_1/mymodel.h5")

In [24]:
def predict_next_word(input_text,n_best):
    input_text=input_text.lower()
    X=np.zeros((1,n_words,len(unique_token)))
    for i , word in enumerate(input_text.split()):
        X[0,i,unique_token_index[word]]=1

    predictions=model.predict(X)[0]
    return np.argpartition(predictions,-n_best)[-n_best:]

In [25]:
possible=predict_next_word("I know that the only person is",5)

1/1 [==============================] - 1s 650ms/step


In [26]:
[unique_token[idx] for idx in possible]

['upon', 'with', 'we', 'any', 'as']

In [27]:
def Generate_text(input_text,text_length,creativity=3): 
    #creativity is how many do we want to chose from 
    #like what the best word will be from the n_best choices
    word_sequence=input_text.split()
    current=0
    for _ in range(text_length):
        sub_sequence=' '.join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice=unique_token[random.choice(list(predict_next_word(sub_sequence,creativity)))]
        except:
            choice=random.choice(unique_token)
        word_sequence.append(choice)
        current+=1
    return " ".join(word_sequence)

In [34]:
Final_text=Generate_text("I know that the only person is",115,5)

1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 5ms/step


In [35]:
Final_text

'I know that the only person is upon in upon in a and as as any upon we thee there and there in unto for a upon i his the and that with we and they they with his in a upon and on his in they at with these unto their his his the upon and these toward s there and forth the and there in and out there the in the from and and there the upon his his in the and out their every and in they upon the upon and into these these of in in their in these that we to they in a with we in their in a in a to a it thou he'

In [36]:
import pickle

# Assuming unique_token and unique_token_index are your data
# unique_tokens = ...  # your unique tokens
# unique_token_index = ...  # your unique token index mapping

# Save unique_tokens
with open('C:/Users/RohanRVC/Documents/Business/E-books/E_Books_Unknown_AI/text_generation/Model/model_stage_1/unique_tokens.pkl', 'wb') as f:
    pickle.dump(unique_token, f)

# Save unique_token_index
with open('C:/Users/RohanRVC/Documents/Business/E-books/E_Books_Unknown_AI/text_generation/Model/model_stage_1/unique_token_index.pkl', 'wb') as f:
    pickle.dump(unique_token_index, f)


In [27]:
chunk_size = 100000  # Adjust as per your memory capacity
n_words = 10  # Number of words in each sequence


In [28]:
# Load unique tokens and indices
with open('C:/Users/RohanRVC/Documents/Business/E-books/E_Books_Unknown_AI/text_generation/Model/model_stage_1/unique_tokens.pkl', 'rb') as f:
    unique_tokens = pickle.load(f)

with open('C:/Users/RohanRVC/Documents/Business/E-books/E_Books_Unknown_AI/text_generation/Model/model_stage_1/unique_token_index.pkl', 'rb') as f:
    unique_token_index = pickle.load(f)


In [29]:
# Load an existing model or initialize a new one
try:
    model = load_model("C:/Users/RohanRVC/Documents/Business/E-books/E_Books_Unknown_AI/text_generation/Model/model_stage_1/mymodel.h5")
except IOError:
    model = Sequential()  # Add model layers here
    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01), metrics=['accuracy'])


In [30]:
# for i in range(0, len(joined_text), chunk_size):
#     partial_text = joined_text[i:i + chunk_size]
#     tokens = tokenizer.tokenize(partial_text.lower())

#     # Update unique tokens and indices
#     unique_token = np.unique(tokens)
#     combined_tokens = set(unique_tokens).union(set(unique_token))
#     updated_unique_tokens = np.array(list(combined_tokens))
#     updated_unique_token_index = {token: idx for idx, token in enumerate(updated_unique_tokens)}

#     # Check if vocabulary size has changed
#     if len(updated_unique_tokens) != model.input_shape[-1]:
#         # Rebuild and compile the model with the new vocabulary size
#         model = Sequential()  # Add model layers here
#         # Compile the model
#         model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01), metrics=['accuracy'])

#     # Prepare data for model training
#     X = np.zeros((len(input_words), n_words, len(updated_unique_tokens)), dtype=bool)
#     y = np.zeros((len(next_words), len(updated_unique_tokens)), dtype=bool)

#     for i, words in enumerate(input_words):
#         for j, word in enumerate(words):
#             X[i, j, updated_unique_token_index[word]] = 1
#         y[i, updated_unique_token_index[next_words[i]]] = 1

#     # Train the model on the current chunk
#     model.fit(X, y, batch_size=128, epochs=30, shuffle=True)


Epoch 1/30
149/149 [==============================] - 14s 76ms/step - loss: 6.0514 - accuracy: 0.1090
Epoch 2/30
149/149 [==============================] - 13s 85ms/step - loss: 5.4197 - accuracy: 0.1685
Epoch 3/30
149/149 [==============================] - 12s 83ms/step - loss: 4.9878 - accuracy: 0.2024
Epoch 4/30
149/149 [==============================] - 12s 80ms/step - loss: 4.6317 - accuracy: 0.2324
Epoch 5/30
149/149 [==============================] - 11s 76ms/step - loss: 4.3145 - accuracy: 0.2580
Epoch 6/30
149/149 [==============================] - 11s 76ms/step - loss: 3.9960 - accuracy: 0.2888
Epoch 7/30
149/149 [==============================] - 12s 78ms/step - loss: 3.6762 - accuracy: 0.3197
Epoch 8/30
149/149 [==============================] - 11s 75ms/step - loss: 3.3523 - accuracy: 0.3658
Epoch 9/30
149/149 [==============================] - 12s 81ms/step - loss: 3.0243 - accuracy: 0.4086
Epoch 10/30
149/149 [==============================] - 11s 73ms/step - loss: 2.698

Epoch 1/30


TypeError: in user code:

    File "c:\Users\RohanRVC\Documents\Business\E-books\E_Books_Unknown_AI\Book\Lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\RohanRVC\Documents\Business\E-books\E_Books_Unknown_AI\Book\Lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\RohanRVC\Documents\Business\E-books\E_Books_Unknown_AI\Book\Lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\RohanRVC\Documents\Business\E-books\E_Books_Unknown_AI\Book\Lib\site-packages\keras\src\engine\training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\RohanRVC\Documents\Business\E-books\E_Books_Unknown_AI\Book\Lib\site-packages\keras\src\engine\training.py"

In [ ]:
model.save("C:/Users/RohanRVC/Documents/Business/E-books/E_Books_Unknown_AI/text_generation/Model/model_stage_2/mymodel.h5")


In [241]:
# with open('C:/Users/RohanRVC/Documents/Business/E-books/E_Books_Unknown_AI/text_generation/Model/model_stage_1/unique_token_index.pkl', 'wb') as f:
#         pickle.dump(updated_unique_tokens, f)

# with open('C:/Users/RohanRVC/Documents/Business/E-books/E_Books_Unknown_AI/text_generation/Model/model_stage_1/unique_token_index.pkl', 'wb') as f:
#         pickle.dump(updated_unique_token_index, f)

# for i in range(0, len(joined_text), chunk_size):
#     partial_text = joined_text[i:i + chunk_size]
#     tokens = tokenizer.tokenize(partial_text.lower())

#     # Update unique tokens and their indices
#     unique_token = np.unique(tokens)
#     combined_tokens = set(unique_tokens).union(set(unique_token))
#     updated_unique_tokens = np.array(list(combined_tokens))
#     updated_unique_token_index = {token: idx for idx, token in enumerate(updated_unique_tokens)}


#     # Prepare data for model training
#     input_words = []
#     next_words = []
#     for i in range(len(tokens) - n_words):
#         input_words.append(tokens[i:i + n_words])
#         next_words.append(tokens[i + n_words])

#     X = np.zeros((len(input_words), n_words, len(updated_unique_tokens)), dtype=bool)
#     y = np.zeros((len(next_words), len(updated_unique_tokens)), dtype=bool)

#     for i, each_sequence in enumerate(input_words):
#         for j, word in enumerate(each_sequence):
#             X[i, j, updated_unique_token_index[word]] = 1
#         y[i, updated_unique_token_index[next_words[i]]] = 1

#     # Define and compile a new model with the updated vocabulary size
#     model = Sequential()
#     model.add(LSTM(128, input_shape=(n_words, len(updated_unique_tokens)), return_sequences=True))
#     model.add(LSTM(128))
#     model.add(Dense(len(updated_unique_tokens)))
#     model.add(Activation('softmax'))
#     model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01), metrics=['accuracy'])

#     # Fine-tune the model on this chunk
#     model.fit(X, y, batch_size=128, epochs=30, shuffle=True)

# # Save the fine-tuned model
# model.save("C:/Users/RohanRVC/Documents/Business/E-books/E_Books_Unknown_AI/text_generation/Model/model_stage_1/mymodel.h5")

6048

In [33]:
import numpy as np
from nltk.tokenize import RegexpTokenizer

def generate_batches(text, chunk_size, n_words, unique_token_index):
    tokenizer = RegexpTokenizer(r'\w+')
    total_length = len(text)

    for i in range(0, total_length, chunk_size):
        partial_text = text[i:i + chunk_size]
        tokens = tokenizer.tokenize(partial_text.lower())
        input_words = []
        next_words = []

        for j in range(len(tokens) - n_words):
            input_words.append(tokens[j:j + n_words])
            next_words.append(tokens[j + n_words])

        X = np.zeros((len(input_words), n_words, len(unique_token_index)), dtype=bool)
        y = np.zeros((len(input_words), len(unique_token_index)), dtype=bool)

        for j, each_sequence in enumerate(input_words):
            for k, word in enumerate(each_sequence):
                if word in unique_token_index:
                    X[j, k, unique_token_index[word]] = 1
            if next_words[j] in unique_token_index:
                y[j, unique_token_index[next_words[j]]] = 1

        yield X, y


In [34]:
# Load unique tokens and indices
with open('C:/Users/RohanRVC/Documents/Business/E-books/E_Books_Unknown_AI/text_generation/Model/model_stage_1/unique_tokens.pkl', 'rb') as f:
    unique_tokens = pickle.load(f)

with open('C:/Users/RohanRVC/Documents/Business/E-books/E_Books_Unknown_AI/text_generation/Model/model_stage_1/unique_token_index.pkl', 'rb') as f:
    unique_token_index = pickle.load(f)


In [36]:
# Load the model
model = load_model("C:/Users/RohanRVC/Documents/Business/E-books/E_Books_Unknown_AI/text_generation/Model/model_stage_1/mymodel.h5")  # Replace with your model path
chunk_size = 100000  # Adjust as per your memory capacity
n_words = 10  # Number of words in each sequence
# Calculate steps per epoch
steps_per_epoch = (len(joined_text) // chunk_size) * (chunk_size // (n_words + 1))

# Train the model using the generator
model.fit(generate_batches(joined_text, chunk_size, n_words, unique_token_index),
          steps_per_epoch=steps_per_epoch,
          epochs=30)


Epoch 1/30


InvalidArgumentError: Graph execution error:

Detected at node 'while/MatMul' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "c:\Users\RohanRVC\Documents\Business\E-books\E_Books_Unknown_AI\Book\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\RohanRVC\Documents\Business\E-books\E_Books_Unknown_AI\Book\Lib\site-packages\traitlets\config\application.py", line 1046, in launch_instance
      app.start()
    File "c:\Users\RohanRVC\Documents\Business\E-books\E_Books_Unknown_AI\Book\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
      self.io_loop.start()
    File "c:\Users\RohanRVC\Documents\Business\E-books\E_Books_Unknown_AI\Book\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\RohanRVC\AppData\Local\Programs\Python\Python311\Lib\

In [ ]:
# Save the trained model
model.save('C:/Users/RohanRVC/Documents/Business/E-books/E_Books_Unknown_AI/text_generation/Model/model_stage_1/mymodel.h5')  # Replace with your desired save path


In [31]:
len(unique_tokens)

2299

In [ ]:
model.save('C:/Users/RohanRVC/Documents/Business/E-books/E_Books_Unknown_AI/text_generation/Model/model_stage_1/mymodel.h5')